# TP2
## Grupo 15

Carlos Eduardo Da Silva Machado A96936

Gonçalo Manuel Maia de Sousa A97485

## Problema 2

### Descrição do Problema

### Abordagem do Problema

## Código Python

In [54]:

from pysmt.shortcuts import *
from pysmt.typing import INT
import random as rn
N = 10
p = 0.01
centro = (0,0)

def gera_rand():
   return rn.choices([0,1],[1-p,p],k=1)[0]
    
print(rn.choices([0,1],[1-p,p],k=1)[0])



0


In [ ]:

def declare(i):
    state = dict()
    state['pc'] = Symbol('pc'+str(i),INT)
    for i in range(1,N+1):
        for j in range(1,N+1):
            state['(i,j)'] = Symbol('(i,j)'+str(i),INT)
    return state

def init(state):
    l = list()
    for i in range(1,N+1):
        l.append(Equals(state[f'({i},1)'], Int(gera_rand())))
    for j in range(2,N+1):
        l.append(Equals(state[f'(1,{j}'], Int(gera_rand())))
    
    ic,jc = centro
    
    for i in [ic-1,ic, ic+1]:
        for j in []